# Zero Pressure Gradient Flat Plate

![plate](http://turbmodels.larc.nasa.gov/FlatPlate/Grids/plateBCpic.jpg)
    
#### References
    
http://turbmodels.larc.nasa.gov/flatplate.html

In [ ]:
DATA_DIR='.'
REF_DATA_DIR='.'

In [ ]:
from zutil import analysis
data_dir = DATA_DIR
ref_data_dir = REF_DATA_DIR
analysis.data_init(default_data_dir=DATA_DIR,
                   default_ref_data_dir=REF_DATA_DIR)

case_dict = {'plate_coarse'      : {'label'    : '(34x24 cells) SST/MG/PRECON',
                                    'facearea' : 0.142692,
                                    'scale'    : 816 },
             'plate_medium'      : {'label'    : '(68x48 cells) SST/MG/PRECON',
                                    'facearea' : 0.075823,
                                    'scale'    : 3264 },
             'plate_fine'        : {'label'    : '(136x96 cells) SST/MG/PRECON',
                                    'facearea' : 0.0391421,
                                    'scale'    : 13056 },
             'plate_finer'       : {'label'    : '(272x192 cells) SST/MG/PRECON',
                                    'facearea' : 0.0198947,
                                    'scale'    : 52224 },
             'plate_medium_f100' : {'label'    : '(68x48 cells) SST/MG/PRECON/WF',
                                    'facearea' : 0.075823,
                                    'scale'    : 3264 }}

### Check friction coefficient [0.0024 < c_f < 0.0028]  at x = 0.97

In [ ]:
valid_lower = 0.0024
valid_upper = 0.0028

cf = {}
h = {}   

### Initialise Environment

In [ ]:
from zutil.plot import *
import math

### Get control dictionary

In [ ]:
from zutil.post import get_case_parameters,print_html_parameters

parameters={}

for case_name in case_dict:
    case_param = get_case_parameters(case_name)
    parameters[case_name] = case_param
      

In [ ]:
# Analysis constants
import zutil
reference_area = 2.0
bl_position = 0.97
mach = 0.2 
kappa = 1.402
R = 287.058
temperature = zutil.to_kelvin(540.0)
pressure = 101325 
density = pressure/(R*temperature)
speed_of_sound = math.sqrt(kappa*pressure/density)
u_ref = mach*speed_of_sound 


### Plotting functions

In [ ]:
from zutil.post import get_case_root, get_case_report
from zutil.post import get_csv_data,get_fw_csv_data
from zutil.post import for_each
def plot_theory(ax, filename):
    df = get_fw_csv_data(filename,widths=[16,16,16,16,16])
    ax.plot(df[1], df[0], color='grey', label='$\mathbf{u^{+}=y^{+}}$')    
    ax.plot(df[2], df[0], color='grey', label='Log law')    

def plot_theory_rough(ax, filename, kr_plus):
    df = get_fw_csv_data(filename,widths=[16,16,16,16,16])
    df[2] -= 5.235 - (8.5 - 1.0/0.41 * log(kr_plus))
    ax.plot(df[2], df[0], color='grey', label='Log law - kr+='+str(kr_plus))    

def plot_comparison(ax,filename):
    df = get_fw_csv_data(filename,widths=[16,14],skiprows=2)    
    ax.plot(df[0], df[1], color='red', label='CFL3D (545x385 points)')    
 
def velocity_plot(data,pts,**kwargs):
    ax = kwargs['axis']
    chart_label = kwargs['chart_label']
    ax.loglog(data.GetPointData()['vprof'], pts.GetPoints()[:,2], label=chart_label)    

def eddy_plot(data,pts,**kwargs):
    ax = kwargs['axis']
    chart_label = kwargs['chart_label']
    ax.semilogy(data.GetPointData()['eddy'], pts.GetPoints()[:,2], label=chart_label)    

def plot_velocity_profile(ax,file_root,label,face_area):
    wall = PVDReader( FileName=file_root+'_wall.pvd' )
    wall_clean = CleantoGrid(Input=wall)
    drag = MinMax(Input=wall_clean)
    drag.Operation = "SUM"
    drag.UpdatePipeline()
    drag_client = servermanager.Fetch(drag)
    cd = drag_client.GetCellData().GetArray("frictionforce").GetValue(0)
    wall_slice = Slice(Input=wall, SliceType="Plane" )
    wall_slice.SliceType.Normal = [1.0,0.0,0.0]
    wall_slice.SliceType.Origin = [bl_position, 0.0, 0.0]
    wall_slice.UpdatePipeline()
    wall_slice_client = servermanager.Fetch(wall_slice)
    nu = wall_slice_client.GetCellData().GetArray("nu").GetValue(0)
    utau = wall_slice_client.GetCellData().GetArray("ut").GetValue(0)
    yplus = wall_slice_client.GetCellData().GetArray("yplus").GetValue(0)
    cf    = wall_slice_client.GetCellData().GetArray("frictionforce").GetValue(0)
    wall_vel = wall_slice_client.GetCellData().GetArray("V").GetValue(0)
    wall_vel = 0.0
    symmetry = PVDReader( FileName=file_root+'_symmetry.pvd' )
    symmetry_clean = CleantoGrid(Input=symmetry)
    CellDatatoPointData1 = CellDatatoPointData(Input=symmetry_clean)
    CellDatatoPointData1.PassCellData = 1
    Clip1 = Clip(Input=CellDatatoPointData1, ClipType="Plane" )
    Clip1.ClipType.Normal = [0.0,1.0,0.0]
    Clip1.ClipType.Origin = [0.0, -0.5, 0.0]
    Clip2 = Clip(Input=Clip1, ClipType="Plane" )
    Clip2.ClipType.Normal = [0.0,0.0,1.0]
    Clip2.ClipType.Origin = [0.0, 0.0, 0.05]
    Clip2.Invert = 1
    Slice1 = Slice(Input=Clip2, SliceType="Plane" )
    Slice1.SliceType.Normal = [1.0,0.0,0.0]
    Slice1.SliceType.Origin = [bl_position, 0.0, 0.0]
    Calculator2 = Calculator(Input=Slice1)
    Calculator2.AttributeMode = 'Point Data'
    Calculator2.Function = '(V.iHat - '+ str(wall_vel) +')'
    Calculator2.ResultArrayName = 'vprof'
    Calculator2.UpdatePipeline()
    sorted_line = PlotOnSortedLines(Input=Calculator2)
    sorted_line.UpdatePipeline()
    extract_client = servermanager.Fetch(sorted_line) 
    chart_label = (('$\mathbf{y^{+} = %.2f}$')%(yplus) + (' $\mathbf{C_d=%.6f}$'%(cd/reference_area)) 
                      + (' $\mathbf{C_f=%.5f}$'%(cf/face_area)) + ' ' + label)
    for_each(extract_client,velocity_plot,axis=ax,chart_label=chart_label)

def plot_eddy_profile(ax,file_root,label,face_area):
    wall = PVDReader( FileName=file_root+'_wall.pvd' )
    wall_clean = CleantoGrid(Input=wall)
    drag = MinMax(Input=wall_clean)
    drag.Operation = "SUM"
    drag.UpdatePipeline()
    drag_client = servermanager.Fetch(drag)
    cd = drag_client.GetCellData().GetArray("frictionforce").GetValue(0)
    wall_slice = Slice(Input=wall, SliceType="Plane" )
    wall_slice.SliceType.Normal = [1.0,0.0,0.0]
    wall_slice.SliceType.Origin = [bl_position, 0.0, 0.0]
    wall_slice.UpdatePipeline()
    wall_slice_client = servermanager.Fetch(wall_slice)
    nu = wall_slice_client.GetCellData().GetArray("nu").GetValue(0)
    utau = wall_slice_client.GetCellData().GetArray("ut").GetValue(0)
    yplus = wall_slice_client.GetCellData().GetArray("yplus").GetValue(0)
    cf    = wall_slice_client.GetCellData().GetArray("frictionforce").GetValue(0)
    wall_vel = wall_slice_client.GetCellData().GetArray("V").GetValue(0)
    wall_vel = 0.0
    symmetry = PVDReader( FileName=file_root+'_symmetry.pvd' )
    symmetry_clean = CleantoGrid(Input=symmetry)
    CellDatatoPointData1 = CellDatatoPointData(Input=symmetry_clean)
    CellDatatoPointData1.PassCellData = 1
    Clip1 = Clip(Input=CellDatatoPointData1, ClipType="Plane" )
    Clip1.ClipType.Normal = [0.0,1.0,0.0]
    Clip1.ClipType.Origin = [0.0, -0.5, 0.0]
    Clip2 = Clip(Input=Clip1, ClipType="Plane" )
    Clip2.ClipType.Normal = [0.0,0.0,1.0]
    Clip2.ClipType.Origin = [0.0, 0.0, 0.05]
    Clip2.Invert = 1
    Slice1 = Slice(Input=Clip2, SliceType="Plane" )
    Slice1.SliceType.Normal = [1.0,0.0,0.0]
    Slice1.SliceType.Origin = [bl_position, 0.0, 0.0]
    Calculator2 = Calculator(Input=Slice1)
    Calculator2.AttributeMode = 'Point Data'
    Calculator2.Function = '(V.iHat - '+ str(wall_vel) +')'
    Calculator2.ResultArrayName = 'vprof'
    Calculator2.UpdatePipeline()
    sorted_line = PlotOnSortedLines(Input=Calculator2)
    sorted_line.UpdatePipeline()
    extract_client = servermanager.Fetch(sorted_line) 
    chart_label = (('$\mathbf{y^{+} = %.2f}$')%(yplus) + (' $\mathbf{C_d=%.6f}$'%(cd/reference_area)) 
                      + (' $\mathbf{C_f=%.5f}$'%(cf/face_area)) + ' ' + label)
    for_each(extract_client,eddy_plot,axis=ax,chart_label=chart_label)
    
def plot_nu(ax,file_root,label,face_area):
    wall = PVDReader( FileName=file_root+'_wall.pvd' )
    wall_clean = CleantoGrid(Input=wall)
    CellDatatoPointData1 = CellDatatoPointData(Input=wall_clean)
    CellDatatoPointData1.PassCellData = 1
    wall_slice = Slice(Input=CellDatatoPointData1, SliceType="Plane" )
    wall_slice.SliceType.Normal = [0.0,1.0,0.0]
    wall_slice.SliceType.Origin = [bl_position, -0.5, 0.0]
    wall_slice.UpdatePipeline()
    wall_slice_client = servermanager.Fetch(wall_slice)
    sorted_line = PlotOnSortedLines(Input=wall_slice)
    sorted_line.UpdatePipeline()
    extract_client = servermanager.Fetch(sorted_line) 
    chart_label = label
    for_each(extract_client,nu_plot,axis=ax,chart_label=chart_label)
    
def bl_plot(data,pts,**kwargs):
    ax = kwargs['axis']
    chart_label = kwargs['chart_label']
    ax.plot(data.GetPointData()['yp'][1:], data.GetPointData()['up'][1:], label=chart_label)    

def get_case_cf(file_root,label,face_area):
    wall = PVDReader( FileName=file_root+'_wall.pvd' )
    wall_clean = CleantoGrid(Input=wall)
    wall_slice = Slice(Input=wall_clean, SliceType="Plane" )
    wall_slice.SliceType.Normal = [1.0,0.0,0.0]
    wall_slice.SliceType.Origin = [bl_position, 0.0, 0.0]
    wall_slice.UpdatePipeline()
    wall_slice_client = servermanager.Fetch(wall_slice)
    cf = wall_slice_client.GetCellData().GetArray("frictionforce").GetValue(0)/face_area
    return(cf) 
    
def plot_profile(ax, file_root, label, kr=0.0):
    wall = PVDReader( FileName=file_root+'_wall.pvd' )
    wall_clean = CleantoGrid(Input=wall)
    wall_slice = Slice(Input=wall_clean, SliceType="Plane" )
    wall_slice.SliceType.Normal = [1.0,0.0,0.0]
    wall_slice.SliceType.Origin = [bl_position, 0.0, 0.0]
    wall_slice.UpdatePipeline()
    wall_slice_client = servermanager.Fetch(wall_slice)
    nu = wall_slice_client.GetCellData().GetArray("nu").GetValue(0)
    utau = wall_slice_client.GetCellData().GetArray("ut").GetValue(0)
    yplus = wall_slice_client.GetCellData().GetArray("yplus").GetValue(0)
    t = wall_slice_client.GetCellData().GetArray("T").GetValue(0)
    p = wall_slice_client.GetCellData().GetArray("p").GetValue(0)
    rho = wall_slice_client.GetCellData().GetArray("rho").GetValue(0)
    wall_vel = wall_slice_client.GetCellData().GetArray("V").GetValue(0)
    symmetry = PVDReader( FileName=file_root+'_symmetry.pvd' )
    symmetry_clean = CleantoGrid(Input=symmetry)
    CellDatatoPointData1 = CellDatatoPointData(Input=symmetry_clean)
    CellDatatoPointData1.PassCellData = 1
    Clip1 = Clip(Input=CellDatatoPointData1, ClipType="Plane" )
    Clip1.ClipType.Normal = [0.0,1.0,0.0]
    Clip1.ClipType.Origin = [0.0, -0.5, 0.0]
    Clip2 = Clip(Input=Clip1, ClipType="Plane" )
    Clip2.ClipType.Normal = [0.0,0.0,1.0]
    Clip2.ClipType.Origin = [0.0, 0.0, 0.05]
    Clip2.Invert = 1
    Slice1 = Slice(Input=Clip2, SliceType="Plane" )
    Slice1.SliceType.Normal = [1.0,0.0,0.0]
    Slice1.SliceType.Origin = [bl_position, 0.0, 0.0]
    Calculator1 = Calculator(Input=Slice1)
    Calculator1.AttributeMode = 'Point Data'
    Calculator1.Function = 'log10(coords.kHat * '+str(utau)+'/'+str(nu)+')'
    Calculator1.ResultArrayName = 'yp'
    Calculator2 = Calculator(Input=Calculator1)
    Calculator2.AttributeMode = 'Point Data'
    Calculator2.Function = '(V.iHat - '+ str(wall_vel) +')/ '+str(utau)
    Calculator2.ResultArrayName = 'up'
    Calculator2.UpdatePipeline()
    sorted_line = PlotOnSortedLines(Input=Calculator2)
    sorted_line.UpdatePipeline()
    extract_client = servermanager.Fetch(sorted_line) 

    # Plot theory for rough surface
    if kr > 0.0:
        plot_theory_rough(ax,os.path.join(analysis.data.ref_data_dir,'data/u+y+theory.csv'),kr*ut/nu);
    chart_label = ('$\mathbf{y^{+}}$ = %.2f '%yplus + ('$\mathbf{u_{T}}$ = %.2f ') %utau + label)
    for_each(extract_client,bl_plot,axis=ax,chart_label=chart_label)

## Plot comparison with Law of the Wall

In [ ]:
from zutil.post import ProgressBar
from zutil.post import get_status_dict
pbar = ProgressBar()
fig = plt.figure(figsize=(10, 5),dpi=100, facecolor='w', edgecolor='#E48B25')

ax = fig.add_subplot(1,1,1)
ax.grid(True)
ax.set_xlabel('$\mathbf{\log(y^{+})}$', fontsize=ft.axis_font_size, fontweight='bold', color = '#5D5858')
ax.set_ylabel('$\mathbf{u^{+}}$', fontsize=ft.axis_font_size, fontweight='bold', color = '#5D5858')
ax.set_title('Zero Pressure Gradient Flat Plate Law of the wall \n', 
             fontsize=ft.title_font_size, 
             fontweight='normal')
ax.axis([0,5,0,30])

for tick in ax.xaxis.get_major_ticks():
    tick.label.set_fontsize(ft.axis_tick_font_size) 
    tick.label.set_fontweight('normal') 
    tick.label.set_color('#E48B25')
for tick in ax.yaxis.get_major_ticks():
    tick.label.set_fontsize(ft.axis_tick_font_size)
    tick.label.set_fontweight('normal') 
    tick.label.set_color('#E48B25')
    
plot_theory(ax,os.path.join(analysis.data.ref_data_dir,'data/u+y+theory.csv'));
plot_comparison(ax,os.path.join(analysis.data.ref_data_dir,'data/flatplate_u+y+_sst.dat'));

for case in case_dict:
    case_name = case
    label = case_dict[case]['label']
    status=get_status_dict(case_name)
    if status:
        num_procs = str(status['num processor'])
        kr = 0.0
        if 'kr' in case_dict[case]:
            kr = case_dict[case]['kr']
        plot_profile(ax, os.path.join(analysis.data.data_dir,get_case_root(case_name,str(num_procs))),label, kr)
    
    pbar+=5
        
legend = ax.legend(loc='best', shadow=False, fontsize=ft.legend_font)
legend.get_frame().set_facecolor('white')

pbar.complete()
plt.show()


## Plot Velocity Profile

In [ ]:
pbar = ProgressBar()
fig = plt.figure(figsize=(10, 5), dpi=100, facecolor='w', edgecolor='#E48B25')
ax = fig.add_subplot(1,1,1)
ax.grid(True)
ax.set_xlabel('speed (m/s)', fontsize=ft.axis_font_size, fontweight='normal', color = '#5D5858')
ax.set_ylabel('$\mathbf{z}$ (m)', fontsize=ft.axis_font_size, fontweight='normal', color = '#5D5858')
ax.set_title('Zero Pressure Gradient Flat Plate Velocity profile \n', fontsize=ft.title_font_size, 
             fontweight='normal')

df = get_fw_csv_data(os.path.join(analysis.data.ref_data_dir,'data/flatplate_u_sst.dat'),widths=[16,14],skiprows=2)    
ax.plot(df[0]*u_ref, df[1], color='red', label='CFL3D (545x385 points)')    

pbar+=5
for case in case_dict:
    case_name = case
    try:
        label = case_dict[case]['label']
        status=get_status_dict(case_name)
        if status:
            num_procs = str(status['num processor'])
            plot_velocity_profile(ax,os.path.join(analysis.data.data_dir,get_case_root(case_name,str(num_procs))),
                              label,case_dict[case]['facearea'])    
    except Exception as e:
        print str(e)
    pbar+=5
    
for tick in ax.xaxis.get_major_ticks():
    tick.label.set_fontsize(ft.axis_tick_font_size) 
    tick.label.set_fontweight('normal') 
    tick.label.set_color('#E48B25')
for tick in ax.yaxis.get_major_ticks():
    tick.label.set_fontsize(ft.axis_tick_font_size)
    tick.label.set_fontweight('normal') 
    tick.label.set_color('#E48B25')    
    
ax.legend(loc='best', shadow=False, fontsize=ft.legend_font)
legend.get_frame().set_facecolor('white')
pbar+=5
pbar.complete()
plt.show()


## Eddy Plot

In [ ]:
pbar = ProgressBar()
fig = plt.figure(figsize=(10, 5), dpi=100, facecolor='w', edgecolor='#E48B25')
ax = fig.add_subplot(1,1,1)
ax.grid(True)
ax.set_xlabel('eddy viscosity ratio', fontsize=ft.axis_font_size, fontweight='normal', color = '#5D5858')
ax.set_ylabel('$\mathbf{z}$ (m)', fontsize=ft.axis_font_size, fontweight='normal', color = '#5D5858')
ax.set_title('Zero Pressure Gradient Flat Plate Eddy profile \n', fontsize=ft.title_font_size, 
             fontweight='normal')

pbar+=5
for case in case_dict:
    case_name = case
    try:
        label = case_dict[case]['label']
        status=get_status_dict(case_name)
        if status:
            num_procs = str(status['num processor'])
            plot_eddy_profile(ax,os.path.join(analysis.data.data_dir,get_case_root(case_name,str(num_procs))),
                              label,case_dict[case]['facearea'])
    except Exception as e:
        print str(e)
    pbar+=5
    
for tick in ax.xaxis.get_major_ticks():
    tick.label.set_fontsize(ft.axis_tick_font_size) 
    tick.label.set_fontweight('normal') 
    tick.label.set_color('#E48B25')
for tick in ax.yaxis.get_major_ticks():
    tick.label.set_fontsize(ft.axis_tick_font_size)
    tick.label.set_fontweight('normal') 
    tick.label.set_color('#E48B25')    
    
ax.legend(loc='best', shadow=False, fontsize=ft.legend_font)
legend.get_frame().set_facecolor('white')
pbar+=5
pbar.complete()
plt.show()

## Mesh Convergence

In [ ]:
pbar = ProgressBar()
fig = plt.figure(figsize=(10, 5),dpi=100, facecolor='w', edgecolor='#E48B25')
ax = fig.add_subplot(1,1,1)

ax.grid(True)
ax.set_xlabel('$\mathbf{h=(1/N)^{1/2}}$', fontsize=ft.axis_font_size, fontweight='bold', color = '#5D5858')
ax.set_ylabel('$\mathbf{C_f}$ at $\mathbf{x=0.97}$', fontsize=ft.axis_font_size, fontweight='bold', color = '#5D5858')
ax.set_title('Zero Pressure Gradient Flat Plate Mesh Convergence ($\mathbf{N}$ = $\mathbf{n}$ x $\mathbf{m}$) \n',
             fontsize=ft.title_font_size, 
             fontweight='normal')

df = get_fw_csv_data(os.path.join(analysis.data.ref_data_dir,'data/cf_convergence_sst_fun3d.dat'),widths=[8,13,12,21],skiprows=3) 
ax.plot(df[2], df[3], color='red', marker='x',label='FUN3D SST')    
df = get_fw_csv_data(os.path.join(analysis.data.ref_data_dir,'data/cf_convergence_sst_cfl3d.dat'),widths=[8,13,12,21],skiprows=3) 
ax.plot(df[2], df[3], color='blue', marker='x',label='CFL3D SST')

pbar+=5

num = 0
cf_plot = []
h_plot = []
for case in case_dict:
    num = num + 1
    case_name = case
    if case_name.endswith('f100'):
        continue
    try:
        status=get_status_dict(case_name)
        if status:
            num_procs = str(status['num processor'])          
            try:
                cf_plot.append(get_case_cf(os.path.join(analysis.data.data_dir,get_case_root(case_name,str(num_procs))),
                        label,case_dict[case]['facearea']))
                h_plot.append(1.0/math.sqrt(case_dict[case]['scale']))
            except:
                pass
    except Exception as e:
        print str(e)
    pbar+=5
ax.plot(h_plot, cf_plot, marker='o', color='#E48B25',label='zCFD SST') 
ax.legend(loc='best', shadow=False, fontsize=ft.legend_font)
legend.get_frame().set_facecolor('white')
pbar+=5

for tick in ax.xaxis.get_major_ticks():
    tick.label.set_fontsize(ft.axis_tick_font_size) 
    tick.label.set_fontweight('normal') 
    tick.label.set_color('#E48B25')
for tick in ax.yaxis.get_major_ticks():
    tick.label.set_fontsize(ft.axis_tick_font_size)
    tick.label.set_fontweight('normal')
    tick.label.set_color('#E48B25')  

pbar.complete()
plt.show()



## Create Eddy Viscosity Images 

In [ ]:
def save_image(file_root, label):
    renderView1 = CreateView('RenderView')
    renderView1.ViewSize = [1281, 993]
    renderView1.InteractionMode = '2D'
    renderView1.AxesGrid = 'GridAxes3DActor'
    renderView1.OrientationAxesVisibility = 0
    renderView1.CenterOfRotation = [1.90, -0.5, 0.0364]
    renderView1.StereoType = 0
    renderView1.CameraPosition = [0.832, -5.77, 0.388]
    renderView1.CameraFocalPoint = [0.832, -0.5, 0.388]
    renderView1.CameraViewUp = [0.0, 0.0, 1.0]
    renderView1.CameraParallelScale = 0.9
    renderView1.Background = [0.0, 0.0, 0.0]
    plate_medium_symmetrypvd = PVDReader(FileName=file_root+'_symmetry.pvd')
    cellDatatoPointData1 = CellDatatoPointData(Input=plate_medium_symmetrypvd)
    eddyLUT = GetColorTransferFunction('eddy')
    eddyLUT.RGBPoints = [1e-07, 0.233, 0.300, 0.750, 
                         196.0, 0.865, 0.865, 0.865, 
                         392.0, 0.700, 0.016, 0.150]
    eddyLUT.ScalarRangeInitialized = 1.0
    cellDatatoPointData1Display = Show(cellDatatoPointData1, renderView1)
    cellDatatoPointData1Display.ColorArrayName = ['POINTS', 'eddy']
    cellDatatoPointData1Display.LookupTable = eddyLUT
    Render()
    WriteImage(label+'.png')
    
from IPython.display import Image, display

for case in case_dict:
    case_name = case
    label_ = case_dict[case]['label']
    label_ = case_name
    status=get_status_dict(case_name)
    if status:
        num_procs = str(status['num processor'])
        save_image(os.path.join(analysis.data.data_dir,get_case_root(case_name,str(num_procs))),label_)
        display(Image(filename=label_+'.png', width=800, height=500, unconfined=True))

            

<script type="text/javascript">
     show=true;
     function toggle(){
        if (show){
             $('div.input').hide();
         }else{
             $('div.input').show();
         }
         show = !show
     }
</script>
<a href="javascript:toggle()" target="_self">toggle input</a>